In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown,display

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
system_prompt="you are an assistant"

In [ ]:
user_prompt=input()

In [ ]:
response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ]
    )
result = response.choices[0].message.content
display(Markdown(result))